In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# threshold = 20 # Anything that occurs less than this will be removed.
# value_counts = train.stack().value_counts() # Entire DataFrame 
# to_remove = value_counts[value_counts <= threshold].index
# train_filtered = train.replace(to_remove, np.nan, inplace=True)
# train.fillna(0,inplace=True)
# train.head()
# X = train[train.columns[1:]]
# new_X=pd.get_dummies(X, prefix=X.columns,columns=X.columns)

In [2]:
train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
test.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [2]:
all_data = pd.concat([train[train.columns[1:]],test[test.columns[1:]]])
all_data.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
threshold = 20 # Anything that occurs less than this will be removed.
value_counts = all_data.stack().value_counts() # Entire DataFrame 
to_remove = value_counts[value_counts <= threshold].index
all_data_filtered = all_data.replace(to_remove, 0)
all_data_dummies=pd.get_dummies(all_data_filtered, prefix=all_data_filtered.columns,columns=all_data_filtered.columns)
all_data_dummies.head()

,RESOURCE_0,RESOURCE_38,RESOURCE_153,RESOURCE_391,RESOURCE_698,RESOURCE_740,RESOURCE_741,RESOURCE_838,RESOURCE_861,RESOURCE_915,...,ROLE_CODE_143185,ROLE_CODE_144355,ROLE_CODE_149339,ROLE_CODE_150754,ROLE_CODE_153959,ROLE_CODE_159789,ROLE_CODE_163313,ROLE_CODE_163732,ROLE_CODE_169635,ROLE_CODE_247660
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = all_data_dummies[:train.shape[0]]
X_test = all_data_dummies[train.shape[0]:]

In [12]:
X_train.shape

(32769, 3856)

In [13]:
train.shape

(32769, 10)

In [5]:
y = train.ACTION

In [6]:

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier
import numpy as np

RANDOM_SEED = 42

clf1 = KNeighborsClassifier(n_jobs=-1)
clf2 = RandomForestClassifier(random_state=RANDOM_SEED,n_jobs=-1)
clf3 = GaussianNB()
lr = LogisticRegression()

# The StackingCVClassifier uses scikit-learn's check_cv
# internally, which doesn't support a random seed. Thus
# NumPy's random seed need to be specified explicitely for
# deterministic behavior
np.random.seed(RANDOM_SEED)
sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
                            meta_classifier=lr)

sclf.fit(X_train.values,y)

StackingCVClassifier(classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='aut...            oob_score=False, random_state=42, verbose=0, warm_start=False), GaussianNB(priors=None)],
           cv=2,
           meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           shuffle=True, stratify=True, use_features_in_secondary=False,
           use_probas=False, verbose=0)

In [9]:
y_pred = sclf.predict_proba(X_test.values)
submission = pd.read_csv("sampleSubmission.csv")
submission.SalePrice = y_pred_correct
submission.to_csv("submission.csv",index = False)





NameError: name 'y_pred_correct' is not defined

In [14]:
submission.Action = y_pred
submission.to_csv("submission.csv",index = False)

In [24]:
from PNPSKC_clf import PNPSKC_classifier
from sklearn.model_selection import train_test_split
xx_train, xx_test, yy_train, yy_test = train_test_split(X_train, y, test_size=0.5, random_state=42)
# from PNPSKC_clf import PNPSKC_classifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# init a list of classifiers
tuned_params ={}
tuned_params['k'] = [1,2,3]
tuned_params['subsample'] = [0.5,0.6,0.7]
tuned_params['colsample'] = [0.3,0.4,0.5]
my_ensemble_clf = []
for my_params in list(ParameterGrid(tuned_params)):
    name = "pnpskc_1_"+str(my_params['k'])+"_"+str(my_params['subsample'])+"_"+str(my_params['colsample'])
    clf = PNPSKC_classifier(k=my_params['k'],subsample=my_params['subsample'],colsample=my_params['colsample'])
    my_ensemble_clf.append((name,clf))

    name = "pnpskc_2_"+str(my_params['k'])+"_"+str(my_params['subsample'])+"_"+str(my_params['colsample'])
    clf = PNPSKC_classifier(k=my_params['k'],subsample=my_params['subsample'],colsample=my_params['colsample'])
    my_ensemble_clf.append((name,clf))

eclf = VotingClassifier(estimators=my_ensemble_clf,voting='soft',n_jobs=-1)
eclf.fit(xx_train,yy_train)
transformed_xx_test = eclf.transform(xx_test)

0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import RidgeClassifier
my_linear_clf = RidgeClassifier()
my_linear_clf.fit(transformed_xx_test,yy_test)
y_pred = my_linear_clf.decision_function(X_test)
submission = pd.read_csv("sampleSubmission.csv")
submission.Action = y_pred
submission.to_csv("submission_SCE.csv",index = False)

